In [1]:
from agentProcessor import AgentProcessor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def AgentFindNeighbours(df, agentIndex, neighbourIndex, output):
    # Don't count self
    if agentIndex == neighbourIndex:
        return output
    # Find vector
    currX, currY = df.iloc[agentIndex][["x", "y"]]
    x, y = df.iloc[neighbourIndex][["x", "y"]]
    dx, dy = x - currX, y - currY
    direction = ""
    # Classify vector
    if dx < 0 and abs(dy) <= abs(dx):
        direction = "W"
    elif dx > 0 and abs(dy) <= abs(dx):
        direction = "E"
    elif dy < 0 and abs(dx) <= abs(dy):
        direction = "S"
    elif dy > 0 and abs(dx) <= abs(dy):
        direction = "N"
    # Update output
    output[direction] = FindMinVec(df, output.get(direction), neighbourIndex, dx, dy)
    return output

def FindMinVec(df, currClosest, index, dx, dy):
    # Ignore self
    if currClosest is None:
        return (index, dx, dy)
    # Compare to what we have stored
    currClosestIndex, currClosestDx, currClosestDy = currClosest
    if (currClosestDx**2 + currClosestDy**2) < (dx**2 + dy**2):
        return currClosest
    else:
        return (index, dx, dy)

In [3]:
def PostProcessData(inData):
    outData = {}
    outData["agent"] = inData["agent"]
    for direction in ["E", "N"]:
        if direction in inData:
            outData[direction+"_x"] = inData[direction][1]
            outData[direction+"_y"] = inData[direction][2]
    return outData

In [4]:
dirMap = {
    "E" : (10, 0),
    #"W" : (-10, 0),
    "N" : (0, 10),
    #"S" : (0, -10)
}

In [5]:
def CalculateVector(df):
    dx = []
    dy = []
    dPartials  = {}
    for direction in dirMap:
        dPartials[direction + "_dx"] = []
        dPartials[direction + "_dy"] = []

    for d in df.iterrows():
        totalXVector = 0.0
        totalYVector = 0.0
        for direction, targetVector in dirMap.items():
            elemX = d[1][direction+"_x"]
            if not np.isnan(elemX):
                elemY = d[1][direction+"_y"]
                targetX = targetVector[0]
                targetY = targetVector[1]
                diffx = (elemX - targetX)
                diffy = (elemY - targetY)
                if diffx > 1 or diffy > 1:
                    diffx /= 10
                    diffy /= 10
                #diffx = 0.0 if abs(diffx) < 0.2 else diffx
                #diffy = 0.0 if abs(diffy) < 0.2 else diffy
                dPartials[direction + "_dx"].append(diffx)
                dPartials[direction + "_dy"].append(diffy)
                #
                totalXVector += diffx
                totalYVector += diffy
            else:
                dPartials[direction + "_dx"].append(np.nan)
                dPartials[direction + "_dy"].append(np.nan)
        '''
        if abs(totalXVector) < 0.05:
            totalXVector = 0.0
        if abs(totalYVector) < 0.05:
            totalYVector = 0.0
        '''
        dx.append(totalXVector)
        dy.append(totalYVector)
    dv = {
        'dx' : dx,
        'dy' : dy
    }
    dv.update(dPartials)
    dfVector = pd.DataFrame(data=dv)
    return dfVector

In [6]:
a = AgentProcessor(100, 30, AgentFindNeighbours, PostProcessData, CalculateVector)

In [7]:
def posPlot(df):
  fig, ax = plt.subplots(1, 1)
  # Plot Prey
  preyScatter = ax.scatter(df["x"], df["y"])
  ax.set(xlim=(-100, 200), ylim=(-1, 200))
  preyScatter.annotation_names = list(range(10))
  plt.close(fig)
  return fig

In [8]:
index = 0
for i in tqdm(range(500)):
  dfs = a.interpolatedStep(5)
  for df in dfs:
    fig = posPlot(df)
    num = str(index).rjust(4, '0')
    fig.savefig(f'figs/{num}.png')
    index += 1

100%|██████████| 500/500 [07:33<00:00,  1.10it/s]


In [10]:
df

,x,y,E_x,E_y,N_x,N_y,dx,dy,E_dx,E_dy,N_dx,N_dy
0,66.02,46.12,30.04,28.89,0.12,9.96,2.12,2.85,2.00,2.89,0.12,-0.04
1,76.06,65.92,10.00,9.09,0.00,9.09,0.00,-0.00,0.00,0.91,0.00,-0.91
2,66.06,75.01,10.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN
3,66.14,56.08,10.83,1.57,-0.08,9.01,0.00,-0.83,0.08,0.16,-0.08,-0.99
4,66.23,36.41,NaN,NaN,-0.21,9.71,-0.21,-0.29,NaN,NaN,-0.21,-0.29
5,76.97,57.65,19.09,17.36,-0.91,8.27,-0.00,0.01,0.91,1.74,-0.91,-1.73
6,86.06,75.01,10.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN
7,66.06,65.09,10.00,0.83,0.00,9.92,0.00,0.75,0.00,0.83,0.00,-0.08
8,76.06,75.01,10.00,0.00,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN
9,96.06,75.01,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
